In [1]:
!pip install transformers
!pip install datasets
!pip install "ray[default]"
!pip install wandb
!pip install tensorboardX

     |████████████████████████████████| 2.5MB 4.1MB/s 
     |████████████████████████████████| 3.3MB 24.4MB/s 
     |████████████████████████████████| 901kB 36.2MB/s 
     |████████████████████████████████| 266kB 5.1MB/s 
     |████████████████████████████████| 245kB 37.4MB/s 
     |████████████████████████████████| 122kB 39.4MB/s 
     |████████████████████████████████| 51.6MB 100kB/s 
     |████████████████████████████████| 1.3MB 33.5MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 10.1MB 41.1MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 3.1MB 31.4MB/s 
     |████████████████████████████████| 133kB 54.3MB/s 
     |████████████████████████████████| 204kB 50.6MB/s 
     |████████████████████████████████| 296kB 45.2MB/s 
     |████████████████████████████████| 143kB 45.0MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 

In [ ]:
%env WANDB_LOG_MODEL=true
%env WANDB_PROJECT=LSTMClassifier

env: WANDB_LOG_MODEL=true
env: WANDB_PROJECT=LSTMClassifier


In [ ]:
from ray import tune
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import wandb
import os

from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLoggerCallback

import torch.optim as optim
import torch
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Exploring LSTM

### Loading the dataset

In [ ]:
import pickle
from datasets import DatasetDict
from pathlib import Path

In [ ]:
base_path = Path("/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/")
full_directory = base_path / 'DistilBERT_multitask_sentence_tokenized_dataset_embeddings'

In [ ]:
with open(full_directory / 'train_ds.pkl', "rb") as input_file:
  train_set_embeddings = pickle.load(input_file)

with open(full_directory / 'val_ds.pkl', "rb") as input_file:
  val_set_embeddings = pickle.load(input_file)

with open(full_directory / 'test_ds.pkl', "rb") as input_file:
  test_set_embeddings = pickle.load(input_file)

dataset_w_embeddings = DatasetDict({'train': train_set_embeddings, 'validation': val_set_embeddings, 'test': test_set_embeddings})
dataset_w_embeddings

DatasetDict({
    train: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 14074
    })
    validation: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 9872
    })
    test: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 9712
    })
})

In [ ]:
dataset_w_embeddings.set_format('pytorch', columns=['pooled_outputs', 'success_label', 'genre'])

In [ ]:
import numpy as np
from datasets import DatasetDict, Dataset

def get_book_changes_idx(book_titles):
  book_changes_idx = np.where(np.array(book_titles[:-1]) != np.array(book_titles[1:]))[0]
  book_changes_idx += 1
  # book_changes_idx = np.append(book_changes_idx, len(book_titles))
  # book_changes_idx = np.insert(book_changes_idx, 0, 0)
  return book_changes_idx

def convert_to_LSTM_dataset_full(dataset):
  full_ds = {}
  full_ds['train'] = convert_to_LSTM_dataset_sub(dataset['train'])
  full_ds['validation'] = convert_to_LSTM_dataset_sub(dataset['validation'])
  full_ds['test'] = convert_to_LSTM_dataset_sub(dataset['test'])

  full_ds = DatasetDict({'train': Dataset.from_dict(full_ds['train']), 'validation': Dataset.from_dict(full_ds['validation']), 'test': Dataset.from_dict(full_ds['test'])})
  # full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  return full_ds

def convert_to_LSTM_dataset_sub(dataset):
  ds = {'grouped_pooled_outs': None, 'success_label': None, 'genre': None}

  book_start_idx = get_book_changes_idx(dataset['book_title'])
  book_start_idx_w_end = np.append(book_start_idx, len(dataset['book_title']))
  book_start_idx_w_zero = np.insert(book_start_idx, 0, 0)

  book_lengths = book_start_idx_w_end - np.concatenate((np.array([0]), np.roll(book_start_idx_w_end, 1)[1:]))
  # print(type(dataset['pooled_outputs']))
  book_grouped_embeddings = dataset['pooled_outputs'].split_with_sizes(list(book_lengths))
  # book_grouped_embeddings = torch.stack(dataset['pooled_outputs'].split_with_sizes(list(book_lengths)), dim=0)

  # print(type(book_grouped_embeddings))
  ds['grouped_pooled_outs'] = book_grouped_embeddings
  ds['success_label'] = np.take(dataset['success_label'], book_start_idx_w_zero)
  ds['genre'] = np.take(dataset['genre'], book_start_idx_w_zero)
  return ds

In [ ]:
# len(full_ds['train']['grouped_pooled_outs'][0])

In [ ]:
class RoBERT_Model(nn.Module):

    def __init__(self, layer_size = 100):
        self.layer_size = layer_size
        super(RoBERT_Model, self).__init__()
        self.lstm = nn.LSTM(768, layer_size, num_layers=1, bidirectional=False)
        self.out = nn.Linear(layer_size, 2)

    def forward(self, grouped_pooled_outs):
        """ Define how to performed each call
        Parameters
        __________
        pooled_output: array
            -
        lengt: int
            -
        Returns:
        _______
        -
        """
        # chunks_emb = pooled_out.split_with_sizes(lengt) # splits the input tensor into a list of tensors where the length of each sublist is determined by lengt

        seq_lengths = torch.LongTensor([x for x in map(len, grouped_pooled_outs)]) # gets the length of each sublist in chunks_emb and returns it as an array

        batch_emb_pad = nn.utils.rnn.pad_sequence(grouped_pooled_outs, padding_value=-91, batch_first=True) # pads each sublist in chunks_emb to the largest sublist with value -91
        batch_emb = batch_emb_pad.transpose(0, 1)  # (B,L,D) -> (L,B,D)
        lstm_input = nn.utils.rnn.pack_padded_sequence(batch_emb, seq_lengths, batch_first=False, enforce_sorted=False) # seq_lengths.cpu().numpy()

        packed_output, (h_t, h_c) = self.lstm(lstm_input, )  # (h_t, h_c))
#         output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, padding_value=-91)

        h_t = h_t.view(-1, self.layer_size) # (-1, 100)

        return self.out(h_t) # logits

In [ ]:
def my_collate1(batches):
  # for batch in batches:
  #   print(type(batch['grouped_pooled_outs']), len(batch['grouped_pooled_outs']))
  #   print(type(torch.FloatTensor(batch['grouped_pooled_outs'])))
    return {
        'grouped_pooled_outs': [torch.stack(x['grouped_pooled_outs']) for x in batches],
        'success_label': torch.LongTensor([x['success_label'] for x in batches])
    }


In [ ]:
from transformers import AdamW
import time

def load_test_data():
  full_ds = convert_to_LSTM_dataset_full(dataset_w_embeddings)
  full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  testset = full_ds['test']
  return testset

def load_data():
  full_ds = convert_to_LSTM_dataset_full(dataset_w_embeddings)
  full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  trainset = full_ds['train']
  valset = full_ds['validation']
  return trainset, valset

def loss_fun(outputs, targets):
    loss = nn.CrossEntropyLoss()
    return loss(outputs, targets)

def rnn_train_fun1(config, checkpoint_dir='/tmp/LSTMModels'):
  model = RoBERT_Model(config["layer_size"])
  model.train()
  device = "cpu"
  # if torch.cuda.is_available():
  #   device = "cuda:0"
  #   if torch.cuda.device_count() > 1:
  #       model = nn.DataParallel(model)
  # # print(type(model))
  # model.to(device)


  criterion = nn.CrossEntropyLoss()
  # optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
  optimizer=AdamW(model.parameters(), lr=config["lr"])

  trainset, valset = load_data()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=config["batch_size"], collate_fn=my_collate1)
  valloader = torch.utils.data.DataLoader(valset, batch_size=config["batch_size"], collate_fn=my_collate1)

  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0

    for batch_idx, batch in enumerate(trainloader):
      grouped_pooled_outs = batch['grouped_pooled_outs'] # .to(device)
      targets = batch['success_label'] #.to(device)

      optimizer.zero_grad()
      outputs = model(grouped_pooled_outs)
      loss = loss_fun(outputs, targets)
      loss.backward()
      model.float()
      optimizer.step()

      running_loss += loss.item()
      epoch_steps += 1

      if batch_idx % 10 == 9:
        print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                        running_loss / epoch_steps))
        running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      with torch.no_grad():
          for i, data in enumerate(valloader, 0):

              grouped_pooled_outs = data['grouped_pooled_outs'] # .to(device)
              targets = data['success_label'] # .to(device)

              outputs = model(grouped_pooled_outs)
              _, predicted = torch.max(outputs.data, 1)

              all_predictions = np.append(all_predictions, predicted.numpy())
              all_labels = np.append(all_labels, targets.numpy())

              loss = criterion(outputs, targets)
              val_loss += loss.cpu().numpy()
              val_steps += 1

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((model.state_dict(), optimizer.state_dict()), path)

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      tune.report(loss=(val_loss / val_steps), f1=s_f1, precision=s_precision, recall=s_recall)
      # if scheduler:
      #     scheduler.step()
      # losses.append(loss.item())
      # if batch_idx % 640 == 0:
      #     print(f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} secondes ___")
      #     t0 = time.time()

In [ ]:
def test_results(net, device="cpu"):
  testset = load_test_data()
  testloader = torch.utils.data.DataLoader(testset, batch_size=8, collate_fn=my_collate1)

  all_predictions = np.array([])
  all_labels = np.array([])

  net.eval()
  with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        grouped_pooled_outs = data['grouped_pooled_outs'] # .to(device)
        targets = data['success_label'] # .to(device)

        outputs = net(grouped_pooled_outs)
        _, predicted = torch.max(outputs.data, 1)

        all_predictions = np.append(all_predictions, predicted.numpy())
        all_labels = np.append(all_labels, targets.numpy())

  s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

  return {
      'precision': s_precision,
      'recall': s_recall,
      'f1': s_f1
  }

In [ ]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger
from ray.tune.schedulers import ASHAScheduler
from functools import partial

def main(num_samples = 6, max_num_epochs = 15):
  config = {
    "lr": tune.loguniform(5e-4, 5e-2),
    "batch_size": tune.choice([16,32,64]),
    "num_epochs": tune.choice([1,2,3,5]),
    "layer_size": tune.choice([100]),
    "wandb": {
      "project": "LSTMClassifier",
      "api_key": "46cb1981ae15765be5bfb5e7c3257d0315a95a1b",
      "log_config": True
    }
  }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(rnn_train_fun1, checkpoint_dir='/tmp/LSTMModels'),
    config = config,
    resources_per_trial={'gpu': 1},
    metric = 'loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="LSTMClassifier",
        group='raytune_hpsearch',
        api_key="46cb1981ae15765be5bfb5e7c3257d0315a95a1b",
        log_config=True
    )])

  
  best_trial = result.get_best_trial(metric="f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["f1"]))
  
  best_trained_model = RoBERT_Model(best_trial.config['layer_size'])
  device = "cpu"
  # if torch.cuda.is_available():
  #     device = "cuda:0"
      # if gpus_per_trial > 1:
      #     best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)
                        
  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/Models/LSTMModels/yungclassifier1.pt"
  torch.save(best_trained_model.state_dict(), path)
  return test_results(best_trained_model, device)
  #,
  # callbacks=[WandbLoggerCallback(
  #     project="LSTMClassifier",
  #     api_key="46cb1981ae15765be5bfb5e7c3257d0315a95a1b",
  #     log_config=True
  # )])

In [ ]:
test_results = main()

2021-07-11 01:16:22,861	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-11 01:16:24,690	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-07-11 01:16:24,692	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2021-07-11 01:16:38,919	WARNING worker.py:1123 -- Warning: The actor ImplicitFunc has size 208670546 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,batch_size,layer_size,lr,num_epochs
DEFAULT_9d3f2_00000,RUNNING,,16,100,0.00308817,2
DEFAULT_9d3f2_00001,PENDING,,64,100,0.000554946,5
DEFAULT_9d3f2_00002,PENDING,,32,100,0.00368687,3
DEFAULT_9d3f2_00003,PENDING,,32,100,0.0452176,1
DEFAULT_9d3f2_00004,PENDING,,32,100,0.047703,2
DEFAULT_9d3f2_00005,PENDING,,64,100,0.000886404,1


(pid=443) 2021-07-11 01:16:41.943092: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Trial name,status,loc,batch_size,layer_size,lr,num_epochs
DEFAULT_9d3f2_00000,RUNNING,,16,100,0.00308817,2
DEFAULT_9d3f2_00001,PENDING,,64,100,0.000554946,5
DEFAULT_9d3f2_00002,PENDING,,32,100,0.00368687,3
DEFAULT_9d3f2_00003,PENDING,,32,100,0.0452176,1
DEFAULT_9d3f2_00004,PENDING,,32,100,0.047703,2
DEFAULT_9d3f2_00005,PENDING,,64,100,0.000886404,1


Result for DEFAULT_9d3f2_00000:
  date: 2021-07-11_01-16-49
  done: false
  experiment_id: 012bfa1091084dd793b7326a2e9a3037
  f1: 0.6085333333333333
  hostname: 8a51874d9367
  iterations_since_restore: 1
  loss: 0.6698870831414273
  node_ip: 172.28.0.2
  pid: 443
  precision: 0.6443746729461015
  recall: 0.6
  should_checkpoint: true
  time_since_restore: 4.997756242752075
  time_this_iter_s: 4.997756242752075
  time_total_s: 4.997756242752075
  timestamp: 1625966209
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9d3f2_00000
  
(pid=443) saving in checkpoint dir


...

Trial name,status,loc,batch_size,layer_size,lr,num_epochs,iter,total time (s),loss,f1,precision
DEFAULT_9d3f2_00005,RUNNING,172.28.0.2:683,64,100,0.000886404,1,4,38.3014,0.590343,0.725326,0.730604
DEFAULT_9d3f2_00000,TERMINATED,,16,100,0.00308817,2,15,60.9831,0.574839,0.744677,0.745446
DEFAULT_9d3f2_00001,TERMINATED,,64,100,0.000554946,5,8,43.753,0.59249,0.735095,0.749484
DEFAULT_9d3f2_00002,TERMINATED,,32,100,0.00368687,3,4,24.7457,0.613002,0.721688,0.732049
DEFAULT_9d3f2_00003,TERMINATED,,32,100,0.0452176,1,1,8.78194,0.846667,0.652293,0.693448
DEFAULT_9d3f2_00004,TERMINATED,,32,100,0.047703,2,1,10.3282,0.828235,0.631338,0.683773


2021-07-11 01:22:06,224	WARNING util.py:162 -- The `process_trial_save` operation took 2.198 s, which may be a performance bottleneck.


Trial name,status,loc,batch_size,layer_size,lr,num_epochs,iter,total time (s),loss,f1,precision
DEFAULT_9d3f2_00000,TERMINATED,,16,100,0.00308817,2,15,60.9831,0.574839,0.744677,0.745446
DEFAULT_9d3f2_00001,TERMINATED,,64,100,0.000554946,5,8,43.753,0.59249,0.735095,0.749484
DEFAULT_9d3f2_00002,TERMINATED,,32,100,0.00368687,3,4,24.7457,0.613002,0.721688,0.732049
DEFAULT_9d3f2_00003,TERMINATED,,32,100,0.0452176,1,1,8.78194,0.846667,0.652293,0.693448
DEFAULT_9d3f2_00004,TERMINATED,,32,100,0.047703,2,1,10.3282,0.828235,0.631338,0.683773
DEFAULT_9d3f2_00005,TERMINATED,,64,100,0.000886404,1,4,38.3014,0.590343,0.725326,0.730604


2021-07-11 01:22:06,402	INFO tune.py:549 -- Total run time: 341.71 seconds (339.45 seconds for the tuning loop).


Best trial config: {'lr': 0.003088165870998023, 'batch_size': 16, 'num_epochs': 2, 'layer_size': 100, 'wandb': {'project': 'LSTMClassifier', 'api_key': '46cb1981ae15765be5bfb5e7c3257d0315a95a1b', 'log_config': True}}
Best trial final validation loss: 0.5748391998441595
Best trial final validation accuracy: 0.7446773911226799


In [ ]:
test_results

{'f1': 0.6704231587909749,
 'precision': 0.6715780871032374,
 'recall': 0.6862068965517242}

In [ ]:
# train_book_start_idx = get_book_changes_idx(dataset_w_embeddings['train']['book_title'])
# lengt = list(train_book_start_idx[0:8])
# lengt = [a_i - b_i for a_i, b_i in zip(lengt, [0] + lengt[0:7])]
# sample_batch = torch.FloatTensor(dataset_w_embeddings['train']['pooled_outputs'][0:train_book_start_idx[len(lengt)-1]])
# grouped_batch = sample_batch.split_with_sizes(lengt)
# seq_lengths = torch.LongTensor(lengt)
# batch_emb_pad = nn.utils.rnn.pad_sequence(grouped_batch, padding_value=-91, batch_first=True)

# Exploring Prediction with Chunk Embeddings

## Generating Chunk Embeddings

We want to generate chunk embeddings. The way this is done is by feeding the tokenized dataset into the model where each element corresponds to a chunk. Then from the model, we extract the last hidden layer. Assuming 512 tokens per chunk, this means that we will get an output of (512, 768) for each chunk. The chunk embedding can then be created by taking the average along the 512 dimension yielding the final output of (1, 768).

In [ ]:
from datasets import DatasetDict

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/PreTokenized/BERT_NER_512/train_dataset.pkl", "rb") as input_file:
  train_dataset = pickle.load(input_file)

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/PreTokenized/BERT_NER_512/val_dataset.pkl", "rb") as input_file:
  val_dataset = pickle.load(input_file)

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/PreTokenized/BERT_NER_512/test_dataset.pkl", "rb") as input_file:
  test_dataset = pickle.load(input_file)

chunked_encoded_dataset = DatasetDict({'train': train_dataset, 'validation': val_dataset, 'test': test_dataset})
chunked_encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'book_title', 'genre', 'input_ids', 'success_label', 'token_type_ids'],
        num_rows: 14525
    })
    validation: Dataset({
        features: ['attention_mask', 'book_title', 'genre', 'input_ids', 'success_label', 'token_type_ids'],
        num_rows: 10151
    })
    test: Dataset({
        features: ['attention_mask', 'book_title', 'genre', 'input_ids', 'success_label', 'token_type_ids'],
        num_rows: 10002
    })
})

In [ ]:
import wandb
from transformers import BertModel
run = wandb.init(project = 'goodreads_success_predictor', name = 'run2')
artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/mlm_BERT:v0', type='model')
artifact_dir = artifact.download()

# Load your Hugging Face model from that folder
# using the same model class
model = BertModel.from_pretrained(artifact_dir, num_labels=2)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact mlm_BERT:v0, 413.38MB. 3 files... Done. 0:0:0
Some weights of the model checkpoint at ./artifacts/mlm_BERT:v0 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./artifacts/mlm_BERT:

In [ ]:
import torch as th

In [ ]:
train_encoded_dataset = {'attention_mask': th.BoolTensor(chunked_encoded_dataset['train']['attention_mask'][:]), 'input_ids': th.LongTensor(chunked_encoded_dataset['train']['input_ids'][:])}
val_encoded_dataset = {'attention_mask': th.BoolTensor(chunked_encoded_dataset['validation']['attention_mask'][:]), 'input_ids': th.LongTensor(chunked_encoded_dataset['validation']['input_ids'][:])}
test_encoded_dataset = {'attention_mask': th.BoolTensor(chunked_encoded_dataset['test']['attention_mask'][:]), 'input_ids': th.LongTensor(chunked_encoded_dataset['test']['input_ids'][:])}

In [ ]:
train_encoded_dataset

{'attention_mask': tensor([[ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False]]),
 'input_ids': tensor([[  101,  1335,  1103,  ...,     0,     0,     0],
         [  101,  1109,  4890,  ...,     0,     0,     0],
         [  101,   107,  2009,  ...,     0,     0,     0],
         ...,
         [  101,  4081, 11287,  ...,     0,     0,     0],
         [  101,  1188,  1159,  ...,     0,     0,     0],
         [  101,  1220,  2141,  ...,     0,     0,     0]])}

In [ ]:
def get_chunk_embeddings(encoded_dataset, batch_size):
  chunk_embeddings = torch.empty([0,768])
  for i in range(len(encoded_dataset['input_ids'])//batch_size + 1):
    up_to = i*batch_size + batch_size
    if len(encoded_dataset['input_ids']) < up_to:
      up_to = len(encoded_dataset['input_ids'])
    input_ids = encoded_dataset['input_ids'][i*batch_size:up_to]
    attention_mask = encoded_dataset['attention_mask'][i*batch_size:up_to]
    with torch.no_grad():
      embeddings = model.forward(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
      embeddings = embeddings * attention_mask[:,:,None]
      embeddings = embeddings.sum(dim=1)/attention_mask.sum(dim=1)[:,None]
      chunk_embeddings = torch.cat([chunk_embeddings, embeddings],0)
    if i % 50 == 0:
      print(i)
  return chunk_embeddings

In [ ]:
print(len(test_encoded_dataset['input_ids'])//32)

312


In [ ]:
test_embeddings = get_chunk_embeddings(test_encoded_dataset, 32)

0
50
100
150
200
250
300


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

with open('test_dataset.pkl', 'wb') as output_file:
  pickle.dump(test_embeddings, output_file)

folder_id = '1n-wByfRXKLJb1RRstD-UDNa9UhwQwa6C'
# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('test_dataset.pkl')
file.Upload() 

## Putting Embeddings together with Labels

In [ ]:
from datasets import DatasetDict
import pickle

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/ChunkEmbeddings/BERT512/train_dataset.pkl", "rb") as input_file:
  train_chunk_embeddings = pickle.load(input_file)

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/ChunkEmbeddings/BERT512/val_dataset.pkl", "rb") as input_file:
  val_chunk_embeddings = pickle.load(input_file)

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/ChunkEmbeddings/BERT512/test_dataset.pkl", "rb") as input_file:
  test_chunk_embeddings = pickle.load(input_file)

In [ ]:
len(train_chunk_embeddings)

14525

In [ ]:
labeled_chunk_embeddings = {
  'train': {'chunk_embedding': train_chunk_embeddings, 'label': chunked_encoded_dataset['train']['success_label']},
  'validation': {'chunk_embedding': val_chunk_embeddings, 'label': chunked_encoded_dataset['validation']['success_label']},
  'test': {'chunk_embedding': test_chunk_embeddings, 'label': chunked_encoded_dataset['test']['success_label']}
}

NameError: ignored

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

with open('train_chunk_embeddings_labelled.pkl', 'wb') as output_file:
  pickle.dump(labeled_chunk_embeddings['train'], output_file)

folder_id = '1n-wByfRXKLJb1RRstD-UDNa9UhwQwa6C'
# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('train_chunk_embeddings_labelled.pkl')
file.Upload() 

## Loading Labeled Embeddings

In [ ]:
import pickle

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/ChunkEmbeddings/BERT512/train_chunk_embeddings_labeled.pkl", "rb") as input_file:
  train_chunk_embeddings = pickle.load(input_file)

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/ChunkEmbeddings/BERT512/validation_chunk_embeddings_labeled.pkl", "rb") as input_file:
  val_chunk_embeddings = pickle.load(input_file)

with open(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/ChunkEmbeddings/BERT512/test_chunk_embeddings_labeled.pkl", "rb") as input_file:
  test_chunk_embeddings = pickle.load(input_file)

In [ ]:
labeled_chunk_embeddings = {
    'train': train_chunk_embeddings,
    'validation': val_chunk_embeddings,
    'test': test_chunk_embeddings
}

In [3]:
import pickle
from datasets import DatasetDict
with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
  avg_pld_outs_hf_ds = pickle.load(input_file)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
type(avg_pld_outs_hf_ds['train']['meaned_pooled_output'])

## SVM (using avg pooled outputs)

In [2]:
from sklearn import svm
import numpy as np

In [ ]:
cs = np.arange(0.9, 1.5, 0.02).tolist()
best_clf = None
best_score = 0
best_c = None
for c in cs:
  clf = svm.SVC(kernel='rbf', gamma='scale', C=c)
  clf.fit(avg_pld_outs_hf_ds['train']['meaned_pooled_output'], avg_pld_outs_hf_ds['train']['success_label'])
  predictions = clf.predict(avg_pld_outs_hf_ds['validation']['meaned_pooled_output'])
  (_, pred_counts) = np.unique(predictions, return_counts=True)
  val_score = f1_score(avg_pld_outs_hf_ds['validation']['success_label'], predictions, average = 'weighted')
  print('Clf with C = {} obtained val-score of {}'.format(c, val_score))
  if (val_score > best_score):
    best_score = val_score
    best_clf = clf
    best_c = c

print('\nBest C: {}; Val-score: {}'.format(best_c, best_score))
test_predictions = best_clf.predict(avg_pld_outs_hf_ds['test']['meaned_pooled_output'])
test_score = f1_score(avg_pld_outs_hf_ds['test']['success_label'], test_predictions, average = 'weighted')
print('Yields score of {} on test set'.format(test_score))

Clf with C = 0.9 obtained val-score of 0.7674047950892664
Clf with C = 0.92 obtained val-score of 0.7674047950892664
Clf with C = 0.9400000000000001 obtained val-score of 0.7674047950892664
Clf with C = 0.9600000000000001 obtained val-score of 0.7674047950892664
Clf with C = 0.9800000000000001 obtained val-score of 0.7674047950892664
Clf with C = 1.0 obtained val-score of 0.7674047950892664
Clf with C = 1.02 obtained val-score of 0.7674047950892664
Clf with C = 1.04 obtained val-score of 0.7674047950892664
Clf with C = 1.06 obtained val-score of 0.7674047950892664
Clf with C = 1.08 obtained val-score of 0.7674047950892664
Clf with C = 1.1 obtained val-score of 0.7674047950892664
Clf with C = 1.12 obtained val-score of 0.7674047950892664
Clf with C = 1.1400000000000001 obtained val-score of 0.7674047950892664
Clf with C = 1.1600000000000001 obtained val-score of 0.7674047950892664
Clf with C = 1.1800000000000002 obtained val-score of 0.7674047950892664
Clf with C = 1.2000000000000002 ob

In [ ]:
clf = svm.SVC(kernel='rbf', gamma='scale', tol = 1e-5)
clf.fit(avg_pld_outs_hf_ds['train']['meaned_pooled_output'], avg_pld_outs_hf_ds['train']['success_label'])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=1e-05, verbose=False)

In [ ]:
predictions = clf.predict(avg_pld_outs_hf_ds['validation']['meaned_pooled_output'])

In [ ]:
(_, pred_counts) = np.unique(predictions, return_counts=True)

In [ ]:
pred_counts

array([ 78, 212])

In [ ]:
from sklearn.metrics import f1_score, precision_recall_curve, roc_auc_score, roc_curve
f1_score(avg_pld_outs_hf_ds['validation']['success_label'], predictions, average = 'weighted')

0.7674047950892664

In [ ]:
test_predictions = clf.predict(avg_pld_outs_hf_ds['test']['meaned_pooled_output'])
f1_score(avg_pld_outs_hf_ds['test']['success_label'], test_predictions, average = 'weighted')

0.6908241707922791

## SVM (Old Code)

### Raw SVM

In [ ]:
from sklearn import svm
import numpy as np

In [ ]:
clf = svm.SVC()
clf.fit(labeled_chunk_embeddings['train']['chunk_embedding'], labeled_chunk_embeddings['train']['label'])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predictions = clf.predict(labeled_chunk_embeddings['validation']['chunk_embedding'])

In [ ]:
(_, pred_counts) = np.unique(predictions, return_counts=True)

In [ ]:
pred_counts

array([ 439, 9712])

In [ ]:
(unique, true_counts) = np.unique(labeled_chunk_embeddings['validation']['label'], return_counts=True)

In [ ]:
true_counts

array([2814, 7337])

### Fancy SVM

In [ ]:
from sklearn.metrics import f1_score, precision_recall_curve, roc_auc_score, roc_curve

In [ ]:
clf = svm.SVC(class_weight = 'balanced')
clf.fit(labeled_chunk_embeddings['train']['chunk_embedding'], labeled_chunk_embeddings['train']['label'])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predictions = clf.predict(labeled_chunk_embeddings['validation']['chunk_embedding'])

In [ ]:
(_, pred_counts) = np.unique(predictions, return_counts=True)

In [ ]:
pred_counts

array([5027, 5124])

In [ ]:
f1_score(labeled_chunk_embeddings['validation']['label'], predictions, average = 'weighted')

0.6317016079143016

In [ ]:
f1_score(labeled_chunk_embeddings['validation']['label'], [1] * len(labeled_chunk_embeddings['validation']['label']), average = 'weighted')

0.6064822033587431

## Shallow NN

In [ ]:
!pip install keras
!pip install scikit-learn==0.21.2
import keras
import numpy as np

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [ ]:
scaler = StandardScaler()
scaler.fit(labeled_chunk_embeddings['train']['chunk_embedding'])

X_train = scaler.transform(labeled_chunk_embeddings['train']['chunk_embedding'])
X_val = scaler.transform(labeled_chunk_embeddings['validation']['chunk_embedding'])

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[1, 768]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [ ]:
keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

keras_param_space = {"n_hidden": [1, 2, 3, 4],
                      "n_neurons": np.arange(30, 300),
                      "learning_rate": reciprocal(3e-4, 3e-2) 
}

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

keras_rand_search = RandomizedSearchCV(keras_clf, keras_param_space, n_iter=20, 
                                   cv=5, scoring="f1_weighted", n_jobs=-1, verbose=True)

keras_rand_search.fit(labeled_chunk_embeddings['train']['chunk_embedding'].numpy(), np.array(labeled_chunk_embeddings['train']['label']), epochs=100,
                       validation_data=(labeled_chunk_embeddings['validation']['chunk_embedding'].numpy(), np.array(labeled_chunk_embeddings['validation']['label'])),
                       callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 14.5min


Epoch 1/100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 30.7min finished


454/454 [==============================] - 1s 3ms/step - loss: 0.6954 - accuracy: 0.6920 - val_loss: 0.5944 - val_accuracy: 0.6801
Epoch 2/100
454/454 [==============================] - 1s 2ms/step - loss: 0.5710 - accuracy: 0.7026 - val_loss: 0.5615 - val_accuracy: 0.7211
Epoch 3/100
454/454 [==============================] - 1s 2ms/step - loss: 0.5751 - accuracy: 0.7017 - val_loss: 0.5483 - val_accuracy: 0.7209
Epoch 4/100
454/454 [==============================] - 1s 2ms/step - loss: 0.5712 - accuracy: 0.7019 - val_loss: 0.5494 - val_accuracy: 0.7232
Epoch 5/100
454/454 [==============================] - 1s 2ms/step - loss: 0.5632 - accuracy: 0.7050 - val_loss: 0.5571 - val_accuracy: 0.7231
Epoch 6/100
454/454 [==============================] - 1s 2ms/step - loss: 0.5546 - accuracy: 0.7149 - val_loss: 0.5475 - val_accuracy: 0.7214
Epoch 7/100
454/454 [==============================] - 1s 2ms/step - loss: 0.5537 - accuracy: 0.7102 - val_loss: 0.5407 - val_accuracy: 0.7184
Epoch 8/100

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f217f5f9650>,
                   iid='warn', n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f217f5ff290>,
                                        'n_hidden': [1, 2, 3, 4],
                                        'n_neurons': array([ 30,  31,  32,  33,  34,  35,  36,  3...
       238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
       251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
       264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
       277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289,
       290, 291, 292, 293, 294, 295, 296, 297, 298, 299])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scorin

In [ ]:
keras_rand_search.best_score_

0.6236865966981083

In [ ]:
keras_rand_search.best_params_

{'learning_rate': 0.022833402546283226, 'n_hidden': 4, 'n_neurons': 190}

In [ ]:
keras_rand_search.best_index_

19

In [ ]:
best_estimator = keras_rand_search.best_estimator_

In [ ]:
val_preds = best_estimator.predict(labeled_chunk_embeddings['validation']['chunk_embedding'].numpy())

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
(_, pred_counts) = np.unique(val_preds, return_counts=True)

In [ ]:
pred_counts

array([6533, 3618])

In [ ]:
train_preds = best_estimator.predict(labeled_chunk_embeddings['train']['chunk_embedding'].numpy())

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
(yung, pred_counts) = np.unique(train_preds, return_counts=True)

In [ ]:
pred_counts

array([9030, 5495])

In [ ]:
yung

array([0, 1])

In [ ]:
f1_score(labeled_chunk_embeddings['train']['label'], train_preds, average = 'weighted')

0.4111326708186815

In [ ]:
type(train_preds)

numpy.ndarray

In [ ]:
train_preds = np.where((train_preds == 0)|(train_preds == 1), train_preds^1, train_preds)

In [ ]:
train_preds

array([1, 1, 1, ..., 1, 1, 1])

# Exploring ELECTRA

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="google/electra-base-generator",
    tokenizer="google/electra-base-generator"
)

print(
    fill_mask(f"HuggingFace is creating a {fill_mask.tokenizer.mask_token} that the community uses to solve NLP tasks.")
)


[{'sequence': 'huggingface is creating a tool that the community uses to solve nlp tasks.', 'score': 0.15631671249866486, 'token': 6994, 'token_str': 'tool'}, {'sequence': 'huggingface is creating a system that the community uses to solve nlp tasks.', 'score': 0.09474308788776398, 'token': 2291, 'token_str': 'system'}, {'sequence': 'huggingface is creating a database that the community uses to solve nlp tasks.', 'score': 0.06263694912195206, 'token': 7809, 'token_str': 'database'}, {'sequence': 'huggingface is creating a framework that the community uses to solve nlp tasks.', 'score': 0.053398653864860535, 'token': 7705, 'token_str': 'framework'}, {'sequence': 'huggingface is creating a program that the community uses to solve nlp tasks.', 'score': 0.04601737856864929, 'token': 2565, 'token_str': 'program'}]
